In [1]:
import pandas as pd
import psycopg2

# Load the CSV files
office_df = pd.read_csv('./1.Office.csv')
employee_df = pd.read_csv('./2.Employee.csv')
worksat_df = pd.read_csv('./3.WorksAt.csv')
cost_df = pd.read_csv('./4.Cost.csv')
costitem_df = pd.read_csv('./5.CostItem.csv')
owner_df = pd.read_csv('./6.Owner.csv')
property_df = pd.read_csv('./7.Property.csv')
owns_df = pd.read_csv('./8.Owns.csv')
client_df = pd.read_csv('./9.Client.csv')
address_df = pd.read_csv('./10.Address.csv')
prefers_df = pd.read_csv('./11.Prefers.csv')
appointment_df = pd.read_csv('./12.Appointment.csv')
transaction_df = pd.read_csv('./13.Transaction.csv')
contract_df = pd.read_csv('./14.Contract.csv')
feedback_df = pd.read_csv('./15.Feedback.csv')

In [3]:
# Database connection
conn = psycopg2.connect(
    "dbname=housing_project user=postgres password=123 host=localhost"
)

cur = conn.cursor()

In [5]:
create_table_commands = [
    """
    CREATE TABLE IF NOT EXISTS Address (
        AddressID INT PRIMARY KEY,
        Street VARCHAR(255),
        City VARCHAR(100),
        State VARCHAR(100),
        ZipCode VARCHAR(20),
        Country VARCHAR(100)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS Office (
        OfficeID INT PRIMARY KEY,
        AddressID INT,
        Phone VARCHAR(20),
        FOREIGN KEY (AddressID) REFERENCES Address(AddressID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS Employee (
        EmployeeID INT PRIMARY KEY,
        FirstName VARCHAR(100),
        LastName VARCHAR(100),
        Role VARCHAR(50),
        Salary DECIMAL(15, 2),
        Phone VARCHAR(20),
        Email VARCHAR(100)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS WorksAt (
        EmployeeID INT,
        OfficeID INT,
        PRIMARY KEY (EmployeeID, OfficeID),
        FOREIGN KEY (EmployeeID) REFERENCES Employee(EmployeeID),
        FOREIGN KEY (OfficeID) REFERENCES Office(OfficeID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS CostItem (
        CostItemID INT PRIMARY KEY,
        CostItem VARCHAR(100)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS Cost (
        CostID VARCHAR(50) PRIMARY KEY,
        CostItemID INT,
        CostDescription TEXT,
        Amount DECIMAL(15, 2),
        Time DATE,
        OfficeID INT,
        FOREIGN KEY (CostItemID) REFERENCES CostItem(CostItemID),
        FOREIGN KEY (OfficeID) REFERENCES Office(OfficeID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS Owner (
        OwnerID INT PRIMARY KEY,
        FirstName VARCHAR(100),
        LastName VARCHAR(100),
        Phone VARCHAR(20),
        Email VARCHAR(100),
        AddressID INT,
        FOREIGN KEY (AddressID) REFERENCES Address(AddressID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS Property (
        PropertyID INT PRIMARY KEY,
        Availability VARCHAR(10),
        ListingType VARCHAR(20),
        UnitType VARCHAR(20),
        SalePrice DECIMAL(15, 2),
        LeasePricePerYear DECIMAL(15, 2),
        BuildingYear INT,
        DateAdded DATE,
        AddressID INT,
        FOREIGN KEY (AddressID) REFERENCES Address(AddressID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS Owns (
        OwnerID INT,
        PropertyID INT,
        PRIMARY KEY (OwnerID, PropertyID),
        FOREIGN KEY (OwnerID) REFERENCES Owner(OwnerID),
        FOREIGN KEY (PropertyID) REFERENCES Property(PropertyID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS Client (
        ClientID INT PRIMARY KEY,
        FirstName VARCHAR(100),
        LastName VARCHAR(100),
        DesiredTransactionType VARCHAR(20),
        Phone VARCHAR(20),
        Email VARCHAR(100),
        AddressID INT,
        FOREIGN KEY (AddressID) REFERENCES Address(AddressID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS ClientPropertyPreferences (
        ClientID INT,
        PreferredPropertyType VARCHAR(50),
        Budget DECIMAL(15, 2),
        PRIMARY KEY (ClientID, PreferredPropertyType),
        FOREIGN KEY (ClientID) REFERENCES Client(ClientID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS Appointment (
        AppointmentID VARCHAR(50) PRIMARY KEY,
        ClientID INT,
        PropertyID INT,
        AgentID INT,
        AppointmentMadeDate DATE,
        VisitDate DATE,
        FOREIGN KEY (ClientID) REFERENCES Client(ClientID),
        FOREIGN KEY (PropertyID) REFERENCES Property(PropertyID),
        FOREIGN KEY (AgentID) REFERENCES Employee(EmployeeID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS Transaction (
        TransactionID VARCHAR(50) PRIMARY KEY,
        AgentID INT,
        TransactionDate DATE,
        FOREIGN KEY (AgentID) REFERENCES Employee(EmployeeID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS Contract (
        ContractID VARCHAR(50) PRIMARY KEY,
        TransactionID VARCHAR(50),
        ClientID INT,
        PropertyID INT,
        StartDate DATE,
        EndDate DATE,
        ContractType VARCHAR(20),
        Amount DECIMAL(15, 2),
        ContractStatus VARCHAR(20),
        RenewalOption VARCHAR(50),
        DepositAmount DECIMAL(15, 2),
        MonthlyRent DECIMAL(15, 2),
        TerminationClause TEXT,
        SignedDate DATE,
        WitnessName VARCHAR(100),
        SpecialTerms TEXT,
        FOREIGN KEY (TransactionID) REFERENCES Transaction(TransactionID),
        FOREIGN KEY (ClientID) REFERENCES Client(ClientID),
        FOREIGN KEY (PropertyID) REFERENCES Property(PropertyID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS Feedback (
        FeedbackID VARCHAR(50) PRIMARY KEY,
        ClientID INT,
        AgentID INT,
        Rating INT,
        Feedback TEXT,
        InteractionType VARCHAR(20),
        FeedbackDate DATE,
        FOREIGN KEY (ClientID) REFERENCES Client(ClientID),
        FOREIGN KEY (AgentID) REFERENCES Employee(EmployeeID)
    );
    """
]


In [7]:
for command in create_table_commands:
    cur.execute(command)

In [9]:
# Insert data into Address table
for _, row in address_df.iterrows():
    cur.execute("""
        INSERT INTO Address (AddressID, Street, City, State, ZipCode, Country) VALUES (%s, %s, %s, %s, %s, %s)
    """, (int(row['AddressID']), row['Street'], row['City'], row['State'], row['ZipCode'], row['Country']))

# Insert data into Office table
for _, row in office_df.iterrows():
    cur.execute("""
        INSERT INTO Office (OfficeID, AddressID, Phone) VALUES (%s, %s, %s)
    """, (int(row['OfficeID']), int(row['AddressID']), row['Phone']))

# Insert data into Employee table
for _, row in employee_df.iterrows():
    cur.execute("""
        INSERT INTO Employee (EmployeeID, FirstName, LastName, Role, Salary, Phone, Email) VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (int(row['EmployeeID']), row['FirstName'], row['LastName'], row['Role'], float(row['Salary']), row['Phone'], row['Email']))

# Insert data into WorksAt table
for _, row in worksat_df.iterrows():
    cur.execute("""
        INSERT INTO WorksAt (EmployeeID, OfficeID) VALUES (%s, %s)
    """, (int(row['EmployeeID']), int(row['OfficeID'])))

# Insert data into CostItem table
for _, row in costitem_df.iterrows():
    cur.execute("""
        INSERT INTO CostItem (CostItemID, CostItem) VALUES (%s, %s)
    """, (int(row['CostItemID']), row['CostItem']))

# Insert data into Cost table
for _, row in cost_df.iterrows():
    cur.execute("""
        INSERT INTO Cost (CostID, CostItemID, CostDescription, Amount, Time, OfficeID) VALUES (%s, %s, %s, %s, %s, %s)
    """, (row['CostID'], int(row['CostItemID']), row['CostDescription'], float(row['Amount']), row['Time'], int(row['OfficeID'])))

# Insert data into Owner table
for _, row in owner_df.iterrows():
    cur.execute("""
        INSERT INTO Owner (OwnerID, FirstName, LastName, Phone, Email, AddressID) VALUES (%s, %s, %s, %s, %s, %s)
    """, (int(row['OwnerID']), row['FirstName'], row['LastName'], row['Phone'], row['Email'], int(row['AddressID'])))

# Insert data into Property table
for _, row in property_df.iterrows():
    cur.execute("""
        INSERT INTO Property (PropertyID, Availability, ListingType, UnitType, SalePrice, LeasePricePerYear, BuildingYear, DateAdded, AddressID) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (int(row['PropertyID']), row['Availability'], row['ListingType'], row['UnitType'], float(row['SalePrice']), float(row['LeasePricePerYear']), int(row['BuildingYear']), row['DateAdded'], int(row['AddressID'])))

# Insert data into Owns table
for _, row in owns_df.iterrows():
    cur.execute("""
        INSERT INTO Owns (OwnerID, PropertyID) VALUES (%s, %s)
    """, (int(row['OwnerID']), int(row['PropertyID'])))

# Insert data into Client table
for _, row in client_df.iterrows():
    cur.execute("""
        INSERT INTO Client (ClientID, FirstName, LastName, DesiredTransactionType, Phone, Email, AddressID) VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (int(row['ClientID']), row['FirstName'], row['LastName'], row['DesiredTransactionType'], row['Phone'], row['Email'], int(row['AddressID'])))

# Insert data into ClientPropertyPreferences table
for _, row in prefers_df.iterrows():
    cur.execute("""
        INSERT INTO ClientPropertyPreferences (ClientID, PreferredPropertyType, Budget) VALUES (%s, %s, %s)
    """, (int(row['ClientID']), row['PreferredPropertyType'], float(row['Budget'])))

# Insert data into Appointment table
for _, row in appointment_df.iterrows():
    cur.execute("""
        INSERT INTO Appointment (AppointmentID, ClientID, PropertyID, AgentID, AppointmentMadeDate, VisitDate) VALUES (%s, %s, %s, %s, %s, %s)
    """, (row['AppointmentID'], int(row['ClientID']), int(row['PropertyID']), int(row['AgentID']), row['AppointmentMadeDate'], row['VisitDate']))

# Insert data into Transaction table
for _, row in transaction_df.iterrows():
    cur.execute("""
        INSERT INTO Transaction (TransactionID, AgentID, TransactionDate) VALUES (%s, %s, %s)
    """, (row['TransactionID'], int(row['AgentID']), row['TransactionDate']))

# Insert data into Feedback table
for _, row in feedback_df.iterrows():
    cur.execute("""
        INSERT INTO Feedback (FeedbackID, ClientID, AgentID, Rating, Feedback, InteractionType, FeedbackDate) VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (row['FeedbackID'], int(row['ClientID']), int(row['AgentID']), int(row['Rating']), row['Feedback'], row['InteractionType'], row['FeedbackDate']))


In [13]:
from datetime import datetime
def convert_date(date_str):
    return datetime.strptime(date_str, '%Y-%m-%d') if pd.notna(date_str) else None

# Insert data into Contract table
for _, row in contract_df.iterrows():
    cur.execute("""
        INSERT INTO Contract (ContractID, TransactionID, ClientID, PropertyID, StartDate, EndDate, ContractType, Amount, ContractStatus, RenewalOption, DepositAmount, MonthlyRent, TerminationClause, SignedDate, WitnessName, SpecialTerms) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (row['ContractID'], row['TransactionID'], row['ClientID'], row['PropertyID'], row['StartDate'], convert_date(row['EndDate']), row['ContractType'], row['Amount'], row['ContractStatus'], row['RenewalOption'], row['DepositAmount'], row['MonthlyRent'], row['TerminationClause'], row['SignedDate'], row['WitnessName'], row['SpecialTerms']))


In [17]:
conn.commit()

In [19]:
# Close the cursor and connection
cur.close()
conn.close()